# 文本预处理
:label:`sec_text_preprocessing`

对于序列数据处理问题，我们在 :numref:`sec_sequence`中
评估了所需的统计工具和预测时面临的挑战。
这样的数据存在许多种形式，文本是最常见例子之一。
例如，一篇文章可以被简单地看作一串单词序列，甚至是一串字符序列。
本节中，我们将解析文本的常见预处理步骤。
这些步骤通常包括：

1. 将文本作为字符串加载到内存中。
1. 将字符串拆分为词元（如单词和字符）。
1. 建立一个词表，将拆分的词元映射到数字索引。
1. 将文本转换为数字索引序列，方便模型操作。


In [10]:
import collections
import re
from d2l import torch as d2l

## 读取数据集

首先，我们从H.G.Well的[时光机器](https://www.gutenberg.org/ebooks/35)中加载文本。
这是一个相当小的语料库，只有30000多个单词，但足够我们小试牛刀，
而现实中的文档集合可能会包含数十亿个单词。
下面的函数(**将数据集读取到由多条文本行组成的列表中**)，其中每条文本行都是一个字符串。
为简单起见，我们在这里忽略了标点符号和字母大写。


In [11]:
#@save
# 定义了数据集的URL，可以通过该URL下载数据集。
d2l.DATA_HUB['time_machine'] = (d2l.DATA_URL + 'timemachine.txt',
                                '090b5e7e70c295757f55df93cb0a180b9691891a')

# 用于加载数据集并将每行文本进行预处理。它使用d2l.download()函数下载数据集文件，并将文本行读取到一个列表中。
def read_time_machine():  #@save
    """将时间机器数据集加载到文本行的列表中"""
    with open(d2l.download('time_machine'), 'r') as f:
        lines = f.readlines()
    return [re.sub('[^A-Za-z]+', ' ', line).strip().lower() for line in lines]

# 加载数据集并将文本行存储在lines列表中。
lines = read_time_machine()
# 打印文本总行数。
print(f'# 文本总行数: {len(lines)}')
# 打印第一行文本。
print(lines[0])
# 打印第11行文本。
print(lines[10])
# print(lines)
# 过这段代码，我们可以加载并查看预处理后的《时间机器》文本数据集的内容。

# 文本总行数: 3221
the time machine by h g wells
twinkled and his usually pale face was flushed and animated the


## 词元化

下面的`tokenize`函数将文本行列表（`lines`）作为输入，
列表中的每个元素是一个文本序列（如一条文本行）。
[**每个文本序列又被拆分成一个词元列表**]，*词元*（token）是文本的基本单位。
最后，返回一个由词元列表组成的列表，其中的每个词元都是一个字符串（string）。


In [12]:
# 定义了tokenize()函数，它接受两个参数：lines表示待拆分的文本行列表，token表示词元类型，默认为'word'。
def tokenize(lines, token='word'):  #@save
    """将文本行拆分为单词或字符词元"""
    # 如果token为'word'，则将每行文本通过空格进行拆分，得到一个列表，其中每个元素为一个单词列表。
    if token == 'word':
        return [line.split() for line in lines]
    # 如果token为'char'，则将每行文本拆分为一个字符列表，其中每个字符作为一个元素。
    elif token == 'char':
        # list(line)将每行文本line转换为一个字符列表，其中每个字符作为一个元素。
        # for line in lines遍历文本行列表lines，对每行文本执行上述操作。
        return [list(line) for line in lines]
    # 如果token既不是'word'也不是'char'，则打印错误信息。
    else:
        print('错误：未知词元类型：' + token)

tokens = tokenize(lines)
for i in range(11):
    print(tokens[i])

['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
[]
[]
[]
[]
['i']
[]
[]
['the', 'time', 'traveller', 'for', 'so', 'it', 'will', 'be', 'convenient', 'to', 'speak', 'of', 'him']
['was', 'expounding', 'a', 'recondite', 'matter', 'to', 'us', 'his', 'grey', 'eyes', 'shone', 'and']
['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']


## 词表

词元的类型是字符串，而模型需要的输入是数字，因此这种类型不方便模型使用。
现在，让我们[**构建一个字典，通常也叫做*词表*（vocabulary），
用来将字符串类型的词元映射到从$0$开始的数字索引中**]。
我们先将训练集中的所有文档合并在一起，对它们的唯一词元进行统计，
得到的统计结果称之为*语料*（corpus）。
然后根据每个唯一词元的出现频率，为其分配一个数字索引。
很少出现的词元通常被移除，这可以降低复杂性。
另外，语料库中不存在或已删除的任何词元都将映射到一个特定的未知词元“&lt;unk&gt;”。
我们可以选择增加一个列表，用于保存那些被保留的词元，
例如：填充词元（“&lt;pad&gt;”）；
序列开始词元（“&lt;bos&gt;”）；
序列结束词元（“&lt;eos&gt;”）。


In [13]:
# 这段代码定义了一个名为Vocab的类，用于构建文本词表。
class Vocab:  #@save
    """文本词表"""
    # 初始化词表对象。接受一个词元列表tokens、最小词频min_freq和保留词元列表reserved_tokens作为输入。
    # 根据词频对词元进行排序，并构建词元到索引和索引到词元的映射。
    # tokens=None：接受一个词元列表作为输入，默认为None。
    # min_freq=0：最小词频阈值，默认为0，表示不过滤词频。
    # reserved_tokens=None：保留的词元列表，默认为None，表示没有保留词元。
    def __init__(self, tokens=None, min_freq=0, reserved_tokens=None):
        # 如果tokens为None，将其初始化为空列表。
        if tokens is None:
            tokens = []
        # 如果reserved_tokens为None，将其初始化为空列表。
        if reserved_tokens is None:
            reserved_tokens = []
        # 按出现频率排序
        # 调用count_corpus函数统计词元的频率。
        counter = count_corpus(tokens)
        # 对词频进行排序，从高到低。
        self._token_freqs = sorted(counter.items(), key=lambda x: x[1],
                                   reverse=True)
        # 未知词元的索引为0
        # 将未知词元<unk>与保留词元列表拼接，构建索引到词元的映射。
        self.idx_to_token = ['<unk>'] + reserved_tokens
        # 构建词元到索引的映射。
        self.token_to_idx = {token: idx
                             for idx, token in enumerate(self.idx_to_token)}
        # 遍历词频列表。
        for token, freq in self._token_freqs:
            # 如果词频低于最小词频阈值，则终止循环。
            if freq < min_freq:
                break
            # 如果词元不在词表中，则将其添加到索引到词元的映射和词元到索引的映射中。
            if token not in self.token_to_idx:
                self.idx_to_token.append(token)
                self.token_to_idx[token] = len(self.idx_to_token) - 1

    # 返回词表中词元的数量。通过调用len(vocab)可以获取词表的大小。
    def __len__(self):
        return len(self.idx_to_token)

    # 根据词元获取对应的索引。
    # 当传入的tokens为单个词元时，返回该词元在词表中的索引；
    # 当传入的tokens为词元列表时，返回每个词元在词表中的索引列表。
    def __getitem__(self, tokens):
        if not isinstance(tokens, (list, tuple)):
            return self.token_to_idx.get(tokens, self.unk)
        return [self.__getitem__(token) for token in tokens]

    # 根据索引获取对应的词元。
    # 当传入的indices为单个索引时，返回该索引对应的词元；
    # 当传入的indices为索引列表时，返回每个索引对应的词元列表。
    def to_tokens(self, indices):
        if not isinstance(indices, (list, tuple)):
            return self.idx_to_token[indices]
        return [self.idx_to_token[index] for index in indices]

    @property
    # : 返回未知词元的索引，通常为0。未知词元是指在构建词表时未出现在词表中的词元。
    def unk(self):  # 未知词元的索引为0
        return 0

    @property
    # 返回词表中词元的频率，按照频率从高到低排序的列表。
    def token_freqs(self):
        return self._token_freqs

#  统计词元的频率。这个函数接收一个词元列表作为输入，可以是一维或二维列表。
#   如果输入是二维列表，则将其展平为一维列表后统计词元的频率。返回的是一个collections.Counter对象，记录了每个词元及其对应的频率。
def count_corpus(tokens):  #@save
    """统计词元的频率"""
    # 这里的tokens是1D列表或2D列表
    if len(tokens) == 0 or isinstance(tokens[0], list):
        # 将词元列表展平成一个列表
        tokens = [token for line in tokens for token in line]
    return collections.Counter(tokens)

# 这些函数和属性用于操作词表对象，可以根据词元获取索引、根据索引获取词元，统计词元的频率等。通过这些操作，可以方便地处理文本数据中的词元和词表信息。

我们首先使用时光机器数据集作为语料库来[**构建词表**]，然后打印前几个高频词元及其索引。


In [14]:
# 创建了一个词表对象vocab，该词表对象根据提供的词元列表tokens构建词表。
vocab = Vocab(tokens)
# vocab.token_to_idx是一个字典，它将词元映射到对应的索引。
# list(vocab.token_to_idx.items())[:10]将该字典转换为列表，并打印出前10个词元及其索引的键值对。
print(list(vocab.token_to_idx.items())[:10])

[('<unk>', 0), ('the', 1), ('i', 2), ('and', 3), ('of', 4), ('a', 5), ('to', 6), ('was', 7), ('in', 8), ('that', 9)]


现在，我们可以(**将每一条文本行转换成一个数字索引列表**)。


In [15]:
# 在循环中，我们遍历了索引列表[0, 10]，并分别打印了对应索引的文本和使用词表vocab将文本转换为索引后的结果。
for i in [0, 10]:
    # 通过vocab[tokens[i]]，我们可以获取文本tokens[i]在词表中对应的索引。
    print('文本:', tokens[i])
    print('索引:', vocab[tokens[i]])

文本: ['the', 'time', 'machine', 'by', 'h', 'g', 'wells']
索引: [1, 19, 50, 40, 2183, 2184, 400]
文本: ['twinkled', 'and', 'his', 'usually', 'pale', 'face', 'was', 'flushed', 'and', 'animated', 'the']
索引: [2186, 3, 25, 1044, 362, 113, 7, 1421, 3, 1045, 1]


## 整合所有功能

在使用上述函数时，我们[**将所有功能打包到`load_corpus_time_machine`函数中**]，
该函数返回`corpus`（词元索引列表）和`vocab`（时光机器语料库的词表）。
我们在这里所做的改变是：

1. 为了简化后面章节中的训练，我们使用字符（而不是单词）实现文本词元化；
1. 时光机器数据集中的每个文本行不一定是一个句子或一个段落，还可能是一个单词，因此返回的`corpus`仅处理为单个列表，而不是使用多词元列表构成的一个列表。


In [16]:
#  定义了一个函数load_corpus_time_machine，接受一个参数max_tokens，默认值为-1。
def load_corpus_time_machine(max_tokens=-1):  #@save
    """返回时光机器数据集的词元索引列表和词表"""
    # 调用read_time_machine函数，将时光机器数据集的文本行存储在lines变量中。
    lines = read_time_machine()
    # 调用tokenize函数，将文本行拆分为字符词元的列表，存储在tokens变量中。
    tokens = tokenize(lines, 'char')
    #  创建Vocab对象vocab，将字符词元列表作为参数传入，构建词表。
    vocab = Vocab(tokens)
    # 因为时光机器数据集中的每个文本行不一定是一个句子或一个段落，
    # 所以将所有文本行展平到一个列表中
    # 使用词表将字符词元转换为词元索引列表，将结果存储在corpus变量中。
    corpus = [vocab[token] for line in tokens for token in line]
    #  如果max_tokens大于0，则对词元索引列表进行截断，最多保留前max_tokens个词元。
    if max_tokens > 0:
        corpus = corpus[:max_tokens]
    #  返回词元索引列表corpus和词表vocab。
    return corpus, vocab

# 调用load_corpus_time_machine函数，将返回的词元索引列表和词表分别存储在corpus和vocab变量中。
corpus, vocab = load_corpus_time_machine()
# 分别输出词元索引列表corpus和词表vocab的长度。
len(corpus), len(vocab)

(170580, 28)

## 小结

* 文本是序列数据的一种最常见的形式之一。
* 为了对文本进行预处理，我们通常将文本拆分为词元，构建词表将词元字符串映射为数字索引，并将文本数据转换为词元索引以供模型操作。

## 练习

1. 词元化是一个关键的预处理步骤，它因语言而异。尝试找到另外三种常用的词元化文本的方法。
1. 在本节的实验中，将文本词元为单词和更改`Vocab`实例的`min_freq`参数。这对词表大小有何影响？


[Discussions](https://discuss.d2l.ai/t/2094)
